In [9]:
import pandas as pd
import time
import requests
import folium

ModuleNotFoundError: No module named 'folium'

In [11]:
#polystring method
'''
Input boundary list e.g. [[33.930124, -84.737658], [33.969074, -84.698486], [33.950073, -84.665274]]
and then it will format it appropriately for the query syntax

'''
def polystring(boundarylist):
    string = ""
    for i in boundarylist:
        string+=str(i[0]) + " "
        string+=str(i[1]) + " "
    return string[:-1]
#boundarydata method:
'''
Input a boundarylist, a list of list of latitudes and longitudes, e.g. 
[[33.930124, -84.737658], [33.969074, -84.698486], [33.950073, -84.665274]]
and then pick a setting, to get all nodes, relations, or ways in that boundary
outputted as a dataframe.

Nodes = 0
Ways = 1
Relations = 2
'''   
    
def boundarydata(boundarylist,setting):
    boundarystring = polystring(boundarylist)
    overpass_url = "http://overpass-api.de/api/interpreter"
    if setting==0:
        overpass_query = f"[out:json];node(poly:'{boundarystring}');out;"
    elif setting==1:
        overpass_query = f"[out:json];way(poly:'{boundarystring}');out;"
    elif setting==2:
        overpass_query = f"[out:json];relation(poly:'{boundarystring}');out;"
    else:
        print("Invalid Setting")
        return None
    try:
        response = requests.get(overpass_url, params={'data': overpass_query})
        response = response.json()
        df = pd.json_normalize(response['elements'])
    except Exception as e:
        print(e)
        return None
    return df  
#coordinate method
'''
Give a way ID, it returns the average latitude and longitude of that way, taking the average
of the nodes that it consists of
'''
def coordinate(way):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"[out:json];way({way});node(w);out;"
    try:
        response = requests.get(overpass_url, params={'data': overpass_query})
        response = response.json()
        flat=flon=count=0
        for i in (response['elements']):
            flat+=i['lat'] #running total of latitude
            flon+=i['lon'] #running total of longitude
            count+=1 #count of number of pairs 
        if(count==0):
            print("Error",way)
            return (None, None)
    except Exception as e:
        print("Error",way, e)
        return (None, None)
    return flat / count, flon / count
#boundary method
'''
Given a county's ID, like Cobb, it will return the set of latitude and longitudes that define its 
boundary. Note: this may need finetuning and it can be visualized with the next method.
'''
def boundary(county):
    boundaryset = []
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"[out:json];rel({county});out;"
    try:
        response = requests.get(overpass_url, params={'data': overpass_query})
        response = response.json()
        for i in (response['elements'][0]['members']):
            if i['type']== 'way' and i['role']== 'outer':
                coord = coordinate(i['ref'])
                if coord[0]:
                    boundaryset.append(coord)
            time.sleep(1)
    except:
        print("Error", county)
        return None
    return boundaryset

def mapcoordinate(coordinates):
    lattot = 0
    longtot = 0
    numcoord = len(coordinates)
    for i in coordinates:
        lattot += i[0]
        longtot += i[1]
    m = folium.Map(location=[lattot/numcoord, longtot/numcoord], zoom_start=12)
    for coord in coordinates:
        folium.Marker(coord).add_to(m)
    m.save('map.html')

mapcoordinate([33.930124, -84.737658], [33.969074, -84.698486], [33.950073, -84.665274])

TypeError: mapcoordinate() takes 1 positional argument but 3 were given